In [ ]:
import csv
from pathlib import Path
import plotly.graph_objects as go


# Map csv data to variables
path = Path("~/.finances").expanduser() / "historical_transactions" / "2024_sankey_data.csv"
data = {}
with open(path, mode='r') as file:
    reader = csv.DictReader(file)
    for row in reader:
        data[row['item']] = int(row['amount'])


wages_a = data['wages_a'] # Austin
wages_b = data['wages_b'] # Becca
savings_int = data['savings_int']
rh_div = data['rh_div']
rh_gains = data['rh_gains']
rh_costbasis = data['rh_costbasis']
cc_stmt_credits = data['cc_stmt_credits'] # Statement credits for Amazon, AmEx, USBank only

anchor = data['anchor']
tax_ret_2023 = data['tax_ret_2023'] # $30 cost to file
wedding_gift = data['wedding_gift']

emp_match_a = data['emp_match_a']
emp_match_b = data['emp_match_b']
hsa = data['hsa']
fers = data['fers']

rh_disc_tax_fed = (rh_div + rh_gains) * 0.24 + savings_int * 0.15
rh_disc_tax_state = (savings_int + rh_div + rh_gains) * 0.0637
disc_tax_fica = savings_int * (0.062 + 0.0145)
tax_fed = data['tax_fed_wages'] + rh_disc_tax_fed
tax_state = data['tax_state_wages'] + rh_disc_tax_state
tax_fica = data['tax_fica_wages'] + disc_tax_fica

ira_2023_max = 6500
roth_iras = ira_2023_max * 2
roth_tsp = data['roth_tsp']
roth_401k = data['roth_401k']

health_ins_a = data['health_ins_a']
health_ins_b = data['health_ins_b']
fegli = data['fegli']

net_a = data['net_a']
net_b = data['net_b']
net_rh_disc = savings_int + rh_div + rh_gains - (rh_disc_tax_fed + rh_disc_tax_state + disc_tax_fica)

# Housing:
rent = data['rent']
rent_ins = data['rent_ins']
utilities = data['utilities'] # PSEG, Internet, Phone
housing_and_utilities = rent + rent_ins + utilities

# Car (no loans):
car_ins = data['car_ins']
gas = data['gas']
tolls = data['tolls']
car_maint = data['car_maint'] # For one oil change and registration renewal
car_total_no_loans = car_ins + gas + tolls + car_maint

# Loans:
car_loan = data['car_loan']
car_loan_parents = data['car_loan_parents'] # Only 3 payments (1/1, 4/1, 7/1)
wedding_loan = data['wedding_loan'] # 4 payments (1/1, 4/1, 7/1, 10/1)
student_loans = data['student_loans']
loans_total = car_loan + car_loan_parents + wedding_loan + student_loans

# Health:
becca_gyms = data['becca_gyms'] # Assume same price as last year for Base annual membership?
austin_gym = data['austin_gym'] # Assuming monthly w/o switch to annual in Nov.
medical = data['medical'] # Medical/doctor/therapy/dental bills and CVS/Duane Reade/etc charges
health = becca_gyms + austin_gym + medical

# Beauty:
haircuts = data['haircuts']
cosmetics = data['cosmetics']
clothes = data['clothes']
beauty = haircuts + cosmetics + clothes

# Subscriptions and fees:
subscriptions = data['subscriptions'] # Headspace, Amazon, Amazon photos, apple, paramount+, adobe, spotify
cc_fees = data['cc_fees']
subscriptions_and_fees = subscriptions + cc_fees

# Food/groceries/supplies/staples:
groceries_and_home = data['groceries_and_home'] # Includes house supplies, home improvement, etc.
restaurants = data['restaurants']
food_and_supplies = groceries_and_home + restaurants

# Variable discretionary:
activities = data['activities'] # Concerts, bars, NYC activities, liquor store purchases, etc. (excludes travel)
travel = data['travel']
gifts = data['gifts'] # Gifts for anyone other than us two
other_discretionary = data['other_discretionary'] # New headphones, parking tickets, fridge magnets, etc.
variable_discretionary = activities + travel + gifts + other_discretionary

# Excess savings:
net_income = sum((
    net_a,
    net_b,
    net_rh_disc,
    anchor,
    tax_ret_2023,
    wedding_gift,
    rh_costbasis,
    cc_stmt_credits
))
excess_savings = net_income - sum((
    housing_and_utilities,
    car_total_no_loans,
    loans_total,
    health,
    beauty,
    subscriptions_and_fees,
    food_and_supplies,
    variable_discretionary
))
print(f"Excess Savings: ${int(excess_savings):,}")


# Check Gross Income inflows and outflows (should be equal)
gross_in = wages_a + wages_b + savings_int + rh_div + rh_gains
gross_out = sum((
    hsa,
    fers,
    roth_iras,
    roth_tsp,
    roth_401k,
    tax_fed,
    tax_state,
    tax_fica,
    health_ins_a,
    health_ins_b,
    fegli,
    net_a,
    net_b,
    net_rh_disc
))
print(f"Gross In: ${gross_in:,}", "|", f"Gross Out: ${int(gross_out):,}" )


# Predefined colors for sankey diagram links/flows
red = "rgba(255, 0, 0, 0.3)"
blue = "rgba(0, 0, 255, 0.3)"
cyan = "rgba(0, 255, 255, 0.5)"
green = "rgba(0, 180, 0, 0.5)"
gold = "rgba(204, 171, 0, 0.6)"
grey = "rgb(200, 200, 200)"

# List of tuples in the form (source, value, target, color)
flows = [
    # Income Sources
    ("Wages (Austin)", wages_a, "Gross Income", green),
    ("Wages (Becca)", wages_b, "Gross Income", green),
    ("Savings Acc. Interest", savings_int, "Gross Income", green),
    ("RH House Acc. Dividends", rh_div, "Gross Income", green),
    ("RH House Acc. LT Cap. Gains", rh_gains, "Gross Income", green),

    ("ANCHOR", anchor, "Other 'Income'", green),
    ("2023 Tax Return", tax_ret_2023, "Other 'Income'", green),
    ("Late Wedding Gift", wedding_gift, "Other 'Income'", green),
    ("RH House Acc. Cost-Basis Withdrawal", rh_costbasis, "Other 'Income'", green),
    ("CC Rewards Statement Credits", cc_stmt_credits, "Other 'Income'", green),

    # Retirement
    ("Employer Match (TSP & 401k, Trad.)", emp_match_a + emp_match_b, "Pre-Tax Retirement", gold),
    ("Gross Income", hsa + fers, "Pre-Tax Retirement", gold),
    ("Gross Income", roth_iras + roth_tsp + roth_401k, "Post-Tax Retirement (Roth IRAs Included)", gold),

    ("Pre-Tax Retirement", emp_match_a, "TSP (Trad.)", gold),
    ("Pre-Tax Retirement", emp_match_b, "401k (Trad.)", gold),
    ("Pre-Tax Retirement", hsa, "HSA", gold),
    ("Pre-Tax Retirement", fers, "FERS (Pension)", gold),

    ("Post-Tax Retirement (Roth IRAs Included)", roth_iras, "Roth IRAs x2", gold),
    ("Post-Tax Retirement (Roth IRAs Included)", roth_tsp, "Roth TSP", gold),
    ("Post-Tax Retirement (Roth IRAs Included)", roth_401k, "Roth 401k", gold),

    ("TSP (Trad.)", emp_match_a, "Retirement (Total)", gold),
    ("401k (Trad.)", emp_match_b, "Retirement (Total)", gold),
    ("HSA", hsa, "Retirement (Total)", gold),
    ("FERS (Pension)", fers, "Retirement (Total)", gold),
    ("Roth IRAs x2", roth_iras, "Retirement (Total)", gold),
    ("Roth TSP", roth_tsp, "Retirement (Total)", gold),
    ("Roth 401k", roth_401k, "Retirement (Total)", gold),

    # Taxes
    ("Gross Income", tax_fed + tax_state + tax_fica, "Taxes", red),
    ("Taxes", tax_fed, "Federal", red),
    ("Taxes", tax_state, "State", red),
    ("Taxes", tax_fica, "FICA", red),

    # Insurance Premiums
    ("Gross Income", health_ins_a + health_ins_b + fegli, "Insurance Premiums", cyan),
    ("Insurance Premiums", health_ins_b, "Med/Dent/Vis (Becca)", cyan),
    ("Insurance Premiums", health_ins_a, "FEHB/FEDVIP (Austin)", cyan),
    ("Insurance Premiums", fegli, "FEGLI", cyan),

    # Net Income
    ("Gross Income", net_a + net_b + net_rh_disc, "Net Income (Roth IRAs Excluded)", blue),
    ("Other 'Income'", anchor + tax_ret_2023 + wedding_gift + rh_costbasis + cc_stmt_credits, "Net Income (Roth IRAs Excluded)", blue),

    ("Net Income (Roth IRAs Excluded)", housing_and_utilities, "Housing & Utilities", blue),
    ("Net Income (Roth IRAs Excluded)", car_total_no_loans, "Car (Loan Excluded)", blue),
    ("Net Income (Roth IRAs Excluded)", loans_total, "Loans", blue),
    ("Net Income (Roth IRAs Excluded)", health, "Health", blue),
    ("Net Income (Roth IRAs Excluded)", beauty, "Beauty", blue),
    ("Net Income (Roth IRAs Excluded)", subscriptions_and_fees, "Subscriptions & Fees", blue),
    ("Net Income (Roth IRAs Excluded)", food_and_supplies, "Food & Supplies", blue),
    ("Net Income (Roth IRAs Excluded)", variable_discretionary, "General Discretionary", blue),
    ("Net Income (Roth IRAs Excluded)", excess_savings, "Excess Savings", blue),

    ("Housing & Utilities", rent, "Rent", blue),
    ("Housing & Utilities", rent_ins, "Renter's Insurance", blue),
    ("Housing & Utilities", utilities, "Utilities", blue),

    ("Car (Loan Excluded)", car_total_no_loans, "Car Insurance", blue),
    ("Car (Loan Excluded)", gas + tolls + car_maint, "Gas, Tolls & Maintenance", blue),

    ("Loans", car_loan + car_loan_parents, "Car Loan", blue),
    ("Loans", wedding_loan, "Wedding Loan", blue),
    ("Loans", student_loans, "Student Loans", blue),

    ("Health", becca_gyms, "Becca Gym", blue),
    ("Health", austin_gym, "Austin Gym", blue),
    ("Health", medical, "Medical (Post-Tax)", blue),

    ("Beauty", haircuts, "Hair Care", blue),
    ("Beauty", cosmetics, "Cosmetics", blue),
    ("Beauty", clothes, "Clothes", blue),

    ("Subscriptions & Fees", subscriptions, "Subscriptions", blue),
    ("Subscriptions & Fees", cc_fees, "CC Fees", blue),

    ("Food & Supplies", groceries_and_home, "Groceries & Home", blue),
    ("Food & Supplies", restaurants, "Restaurants", blue),

    ("General Discretionary", activities, "Activities", blue),
    ("General Discretionary", travel, "Travel", blue),
    ("General Discretionary", gifts, "Gifts", blue),
    ("General Discretionary", other_discretionary, "Other Discretionary", blue),
]

# Extract unique labels (nodes) and map each label to its index
labels = list(set([item for src, _, tgt, _ in flows for item in (src, tgt)]))
label_to_index = {label: idx for idx, label in enumerate(labels)}

# Map source and target labels to indices using the dictionary
source_indices = [label_to_index[src] for src, _, tgt, _ in flows]
target_indices = [label_to_index[tgt] for src, _, tgt, _ in flows]
values = [value for _, value, _, _ in flows]
colors = [color for _, _, _, color in flows]

# Create Sankey diagram
fig = go.Figure(data=[go.Sankey(
    arrangement="perpendicular",
    node=dict(
        pad=15,
        thickness=30,
        line=dict(color="black", width=1),
        label=labels,
        color=grey
    ),
    link=dict(
        source=source_indices,
        target=target_indices,
        value=values,
        color=colors
    )
)])

fig.update_layout(
    # title_text="Projected 2024 Income and Expenses (Created Using Plotly)",
    height=900,
    width=1535,
    font_size=12
)
fig.show()


In [ ]:
# Simplified, more concise version of the same sankey diagram (cell must be run after the previous one)


# List of tuples in the form (source, value, target, color)
flows = [
    # Income Sources
    ("Wages (Austin)", wages_a, "Gross Income", green),
    ("Wages (Becca)", wages_b, "Gross Income", green),
    ("Other Taxable Income", savings_int + rh_div + rh_gains, "Gross Income", green),

    # Retirement
    ("Employer Match (TSP & 401k, Trad.)", emp_match_a + emp_match_b, "Pre-Tax Retirement (HSA, 401k, TSP, FERS)", gold),
    ("Gross Income", hsa + fers, "Pre-Tax Retirement (HSA, 401k, TSP, FERS)", gold),
    ("Gross Income", roth_iras + roth_tsp + roth_401k, "Post-Tax Retirement (Roth IRAs, 401k, TSP)", gold),

    ("Pre-Tax Retirement (HSA, 401k, TSP, FERS)", emp_match_a + emp_match_b + hsa + fers, "Retirement (Total)", gold),
    ("Post-Tax Retirement (Roth IRAs, 401k, TSP)", roth_iras + roth_tsp + roth_401k, "Retirement (Total)", gold),

    # Taxes
    ("Gross Income", tax_fed + tax_state + tax_fica, "Taxes", red),
    ("Taxes", tax_fed, "Federal", red),
    ("Taxes", tax_state, "State", red),
    ("Taxes", tax_fica, "FICA", red),

    # Insurance Premiums
    ("Gross Income", health_ins_a + health_ins_b + fegli, "Insurance Premiums", cyan),
    ("Insurance Premiums", health_ins_b, "Med/Dent/Vis (Becca)", cyan),
    ("Insurance Premiums", health_ins_a, "FEHB/FEDVIP (Austin)", cyan),
    ("Insurance Premiums", fegli, "FEGLI", cyan),

    # Net Income
    ("Gross Income", net_a + net_b + net_rh_disc, "Net Income (Roth IRAs Excluded)", blue),
    ("Other 'Income'", anchor + tax_ret_2023 + wedding_gift + rh_costbasis + cc_stmt_credits, "Net Income (Roth IRAs Excluded)", green),

    ("Net Income (Roth IRAs Excluded)", housing_and_utilities, "Housing & Utilities", blue),
    ("Net Income (Roth IRAs Excluded)", car_total_no_loans, "Car (Loan Excluded)", blue),
    ("Net Income (Roth IRAs Excluded)", loans_total, "Loans", blue),
    ("Net Income (Roth IRAs Excluded)", health, "Health", blue),
    ("Net Income (Roth IRAs Excluded)", beauty, "Beauty", blue),
    ("Net Income (Roth IRAs Excluded)", subscriptions_and_fees, "Subscriptions & Fees", blue),
    ("Net Income (Roth IRAs Excluded)", food_and_supplies, "Food & Supplies", blue),
    ("Net Income (Roth IRAs Excluded)", variable_discretionary, "General Discretionary", blue),
    ("Net Income (Roth IRAs Excluded)", excess_savings, "Excess Savings", blue),
]

# Extract unique labels (nodes) and map each label to its index
labels = list(set([item for src, _, tgt, _ in flows for item in (src, tgt)]))
label_to_index = {label: idx for idx, label in enumerate(labels)}

# Map source and target labels to indices using the dictionary
source_indices = [label_to_index[src] for src, _, tgt, _ in flows]
target_indices = [label_to_index[tgt] for src, _, tgt, _ in flows]
values = [value for _, value, _, _ in flows]
colors = [color for _, _, _, color in flows]

# Create Sankey diagram
fig = go.Figure(data=[go.Sankey(
    arrangement="perpendicular", # Or set arrangement to "snap"
    node=dict(
        pad=15,
        thickness=30,
        line=dict(color="black", width=1),
        label=labels,
        color=grey
    ),
    link=dict(
        source=source_indices,
        target=target_indices,
        value=values,
        color=colors
    )
)])

fig.update_layout(
    # title_text="Projected 2024 Income and Expenses, Concise/Simplified Version (Created Using Plotly)",
    height=900,
    width=1200,
    font_size=12
)
fig.show()